In [1]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torchinfo import summary

In [2]:
import os
import zipfile
import shutil
import pandas as pd
from pathlib import Path

import requests
from helper_functions import set_seeds
from torchvision.models import resnet18, ResNet18_Weights

In [3]:
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

/Users/karliekaz/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "Mnist_Image_Data"

train_dir = image_path /"Images" / "train"
test_dir = image_path / "Images"/ "test"


In [6]:

#need to sort the data so that each class is separated within the test and train folders 
# TRAIN FOLDER SORT FIRST: 

# Define the paths for your dataset and the destination folder.
dataset_path_train = 'data/Mnist_Image_Data/train.csv'
destination_folder_train = train_dir

# Read the dataset into a DataFrame.
df = pd.read_csv(dataset_path_train)

# Create folders for each label (0-9) if they don't exist.
for label in range(10):
    label_folder = os.path.join(destination_folder_train, str(label))
    if not os.path.exists(label_folder):
        os.makedirs(label_folder)
moved_files = set()
# Iterate through the dataset and move data points to the appropriate folder.
for index, row in df.iterrows():
    feature_value = row['filename']
    label_value = row['label']
   
    # Create the source and destination paths for the data point.
    source_path_train = os.path.join(destination_folder_train, feature_value)
    destination_path_train = os.path.join(destination_folder_train, str(label_value), feature_value)

    if feature_value not in moved_files:
    # Move the data point to the destination folder.
        shutil.move(source_path_train, destination_path_train)
        moved_files.add(feature_value)
    else:
        print(f"Files have already been moved.")


    

FileNotFoundError: [Errno 2] No such file or directory: 'data/Mnist_Image_Data/Images/train/0.png'

In [ ]:
#need to sort the data so that each class is separated within the test and train folders 
# TEST FOLDER SORT: 

# Define the paths for folder and csv 
dataset_path_test = 'data/Mnist_Image_Data/test.csv'
destination_folder_test = test_dir

# Read the dataset
df = pd.read_csv(dataset_path_test)

# Create folders for each label (0-9) if they don't exist
for label in range(10):
    label_folder = os.path.join(destination_folder_test, str(label))
    if not os.path.exists(label_folder):
        os.makedirs(label_folder)

# Iterate through the dataset and move data points to the appropriate folder
for index, row in df.iterrows():
    filename = row['filename']
    label = row['label']
    
    # Create the source and destination paths for the data point.
    source_path_test = os.path.join(destination_folder_test, filename)
    destination_path_test = os.path.join(destination_folder_test, str(label), filename)
    
    # Move the data point to the destination folder.
    shutil.move(source_path_test, destination_path_test)

    

In [7]:
# 1. Get pretrained weights 
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # .DEFAULT = best available weights 
model = torchvision.models.efficientnet_b0(weights=weights).to(device)

# 3. Freeze the base parameters
for parameter in model.parameters():
    parameter.requires_grad = False
    
# 4. Change the classifier head 
class_names= ['0','1','2','3','4','5','6','7','8','9']

output_shape = len(class_names)

torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Recreate the classifier layer and seed it to the target device
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.5, inplace=True), 
    torch.nn.Linear(in_features=256, 
                    out_features=output_shape)).to(device)

In [8]:
# Get the transforms used to create our pretrained weights


auto_transforms = weights.transforms()
print(auto_transforms)

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


In [9]:
# Print a summary using torchinfo 
summary(model=model, 
        input_size=(32, 3, 224, 224), 
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
) 

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, Conv2dNormActivation: 2, Conv2d: 3, BatchNorm2d: 3, SiLU: 3, Sequential: 2, MBConv: 3, Sequential: 4, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, SqueezeExcitation: 5, AdaptiveAvgPool2d: 6, Conv2d: 6, SiLU: 6, Conv2d: 6, Sigmoid: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, Sequential: 2, MBConv: 3, Sequential: 4, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, SqueezeExcitation: 5, AdaptiveAvgPool2d: 6, Conv2d: 6, SiLU: 6, Conv2d: 6, Sigmoid: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, MBConv: 3, Sequential: 4, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchNorm2d: 6, SiLU: 6, SqueezeExcitation: 5, AdaptiveAvgPool2d: 6, Conv2d: 6, SiLU: 6, Conv2d: 6, Sigmoid: 6, Conv2dNormActivation: 5, Conv2d: 6, BatchN

In [10]:
# Setup dataloaders
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=auto_transforms, # perform same data transforms on our own data as the pretrained model
                                                                               batch_size=64) # set mini-batch size 

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x15f1484c0>,
 ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

In [11]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

# Start the timer
from timeit import default_timer as timer 
start_time = timer()

# Setup training and save the results
results = engine.train(model=model,
                       train_dataloader=train_dataloader,
                       test_dataloader=test_dataloader,
                       optimizer=optimizer,
                       loss_fn=loss_fn,
                       epochs=5,
                       device=device)

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

  0%|          | 0/5 [00:53<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x1280 and 256x10)

In [ ]:
#using class from previous MNIST example in info3000 - need to use it to combine the csv file data with the actual picture
# also need to normalize data 
class MNIST():

    def __init__(self, dataset, dir):
        self.dataset = dataset
        self.dir = dir

    def __len__(self):
        return len(self.dataset)


    def __getitem__(self, idx):
        img_name = os.path.join(self.dir, self.dataset.iloc[idx, 0])
        img = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        label = self.dataset.iloc[idx, 1]
        #img = io.imread(img)
        
        #img = img[:,:,:1] # Read a single channel only
    
        #img = img/255.0   # Normalize or Feature Scale data
        #img = np.moveaxis(img,2,0)
        # normalize
        img = cv2.resize(img, (32, 32))
        img = img / 255.0 # Normalize or Feature Scale data
        img = torch.unsqueeze(torch.FloatTensor(img), 0)  
        label = torch.tensor(int(label))  # Convert label to a tensor

        return img, label